In [1]:
from myosuite.utils import gym

MyoSuite:> Registering Myo Envs


Gym has been unmaintained since 2022 and does not support NumPy 2.0 amongst other critical functionality.
Please upgrade to Gymnasium, the maintained drop-in replacement of Gym, or contact the authors of your software and request that they upgrade.
Users of this version of Gym should be able to simply replace 'import gym' with 'import gymnasium as gym' in the vast majority of cases.
See the migration guide at https://gymnasium.farama.org/introduction/migration_guide/ for additional information.
/home/ubuntu/myochallenge_2025eval/.venv/lib/python3.11/site-packages/gymnasium/envs/registration.py:694: UserWarning: WARN: Overriding environment myoArmReachFixed-v0 already in registry.
  logger.warn(f"Overriding environment {new_spec.id} already in registry.")
/home/ubuntu/myochallenge_2025eval/.venv/lib/python3.11/site-packages/gymnasium/envs/registration.py:694: UserWarning: WARN: Overriding environment myoSarcArmReachFixed-v0 already in registry.
  logger.warn(f"Overriding environment {ne

In [23]:
import numpy as np

In [2]:
env = gym.make("myoChallengeTableTennisP1-v0", max_episode_steps=300)

/home/ubuntu/myochallenge_2025eval/.venv/lib/python3.11/site-packages/glfw/__init__.py:917: GLFWError: (65550) b'X11: The DISPLAY environment variable is missing'
  warnings.warn(message, GLFWError)


    MyoSuite: A contact-rich simulation suite for musculoskeletal motor control
        Vittorio Caggiano, Huawei Wang, Guillaume Durandau, Massimo Sartori, Vikash Kumar
        L4DC-2019 | https://sites.google.com/view/myosuite
    


In [4]:
env.reset()
action = env.action_space.sample()
obs, base_reward, terminated, truncated, info = env.step(action)

In [24]:
obs_dict = info['obs_dict']

In [29]:
time = obs_dict["time"] - 0

obs = np.concatenate([
    np.array([time], dtype=np.float32),

    obs_dict["pelvis_pos"],
    obs_dict["body_qpos"],
    obs_dict["body_qvel"],

    obs_dict["ball_pos"],
    obs_dict["ball_vel"],

    obs_dict["paddle_pos"],
    obs_dict["paddle_vel"],
    obs_dict["paddle_ori"],
    obs_dict["padde_ori_err"],

    obs_dict["reach_err"],
    obs_dict["palm_pos"],
    obs_dict["palm_err"],

    obs_dict["touching_info"],
    obs_dict["act"],
], axis=0)

In [32]:
obs_dict.keys()

dict_keys(['time', 'pelvis_pos', 'body_qpos', 'body_qvel', 'ball_pos', 'ball_vel', 'paddle_pos', 'paddle_vel', 'paddle_ori', 'padde_ori_err', 'reach_err', 'palm_pos', 'palm_err', 'touching_info', 'act'])